<a href="https://colab.research.google.com/github/CargofirstQaho/Commodity_analysis_GC/blob/main/rice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip
!pip install --only-binary=:all: opencv-python-headless==4.8.0.76 numpy==1.24.4 pillow==9.5.0 scikit-image==0.21.0
!pip install --only-binary=:all: streamlit==1.38.0 pyngrok==7.2.0 watchdog==4.0.0 requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: Could not find a version that satisfies the requirement numpy==1.24.4 (from versions: 1.26.0, 1.26.1, 1.26.2, 1.26.3, 1.26.4, 2.0.0, 2.0.1, 2.0.2, 2.1.0, 2.1.1, 2.1.2, 2.1.3, 2.2.0, 2.2.1, 2.2.2, 2.2.3, 2.2.4, 2.2.5, 2.2.6, 2.3.0, 2.3.1, 2.3.2, 2.3.3, 2.3.4, 2.3.5)
ERROR: No matching distribution found for numpy==1.24.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 82.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 89.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 95.3 MB/s  0:00:00
  Attempting uninstall: watchdog
    Found existing installation: watchdog 6.0.0
    Uninstalling watchdog-6.0.0:
      Successfully uninstalled watchdog-6.0.0
  Attempting uninstall: tenacity
    Found existing installation: ten

In [ ]:
app_code = r"""
# app.py
import streamlit as st
import numpy as np
import cv2
from PIL import Image
from io import BytesIO

st.set_page_config(page_title='Rice Quality Analyzer', layout='wide')
st.title('Rice Quality Analyzer ')

st.markdown('Counts: total, broken, normal, foreign matter, damaged/discoloured, chalky. Averages reported in mm. Percentages show class share of total detected grains.')

# Sidebar tuning
st.sidebar.header('Tunable thresholds')
broken_ratio = st.sidebar.slider('Broken length ratio (of median)', 0.4, 0.95, 0.75, 0.01)
chalky_percentile = st.sidebar.slider('Chalky intensity percentile (higher = more chalky)', 50, 99, 75, 1)
damaged_percentile = st.sidebar.slider('Damaged/discoloured intensity percentile (lower = more damaged)', 1, 50, 20, 1)
min_area = st.sidebar.slider('Min contour area (px) filter', 10, 200, 50, 1)
foreign_solidity_thresh = st.sidebar.slider('Foreign solidity threshold (lower means more likely foreign)', 0.1, 0.95, 0.55, 0.01)
foreign_area_multiplier = st.sidebar.slider('Foreign area multiplier (relative to median area)', 1.5, 10.0, 5.0, 0.1)
foreign_aspect_ratio = st.sidebar.slider('Foreign extreme aspect ratio', 1.5, 6.0, 3.0, 0.1)


# Image processing utils
def preprocess(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY_INV, 51, 5)
    kernel = np.ones((3,3), np.uint8)
    opened = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)
    dilated = cv2.dilate(opened, kernel, iterations=1)
    return gray, dilated

def extract_grain_contours(binary_mask, min_area_px):
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    filtered = [c for c in contours if cv2.contourArea(c) > float(min_area_px)]
    return filtered

def grain_features(gray, contour):
    area = cv2.contourArea(contour)
    rect = cv2.minAreaRect(contour)
    (cx, cy), (w, h), angle = rect
    length = max(w, h)
    width = min(w, h)
    aspect_ratio = (length + 1.0) / (width + 1.0)
    mask = np.zeros_like(gray)
    cv2.drawContours(mask, [contour], -1, 255, -1)
    mean_intensity = float(cv2.mean(gray, mask=mask)[0])
    hull = cv2.convexHull(contour)
    hull_area = cv2.contourArea(hull) + 1e-6
    solidity = float(area / hull_area)
    polygon = contour.reshape(-1, 2).tolist()
    return {
        "contour": contour,
        "polygon": polygon,
        "area": float(area),
        "length_px": float(length),
        "width_px": float(width),
        "aspect_ratio": float(aspect_ratio),
        "mean_intensity": mean_intensity,
        "solidity": solidity,
        "cx": float(cx),
        "cy": float(cy)
    }

def classify_grains(features_list, broken_ratio, chalky_percentile, damaged_percentile,
                    foreign_solidity_thresh, foreign_area_multiplier, foreign_ar_thresh):
    results = []
    lengths = np.array([f["length_px"] for f in features_list]) if features_list else np.array([0.0])
    median_len = float(np.median(lengths)) if len(lengths) else 0.0
    broken_threshold = broken_ratio * median_len

    intensities = np.array([f["mean_intensity"] for f in features_list]) if features_list else np.array([0.0])
    chalky_threshold = float(np.percentile(intensities, chalky_percentile)) if len(intensities) else 255.0
    damaged_threshold = float(np.percentile(intensities, damaged_percentile)) if len(intensities) else 0.0

    areas = np.array([f["area"] for f in features_list]) if features_list else np.array([0.0])
    median_area = float(np.median(areas)) if len(areas) else 0.0

    for f in features_list:
        is_broken = f["length_px"] < broken_threshold
        is_chalky = f["mean_intensity"] >= chalky_threshold
        is_damaged = f["mean_intensity"] <= damaged_threshold
        is_foreign = (f["solidity"] < foreign_solidity_thresh) or \
                     (f["area"] > foreign_area_multiplier * median_area) or \
                     (f["aspect_ratio"] > foreign_ar_thresh)
        is_normal = (not is_broken) and (not is_chalky) and (not is_damaged) and (not is_foreign)
        label = "normal"
        if is_foreign:
            label = "foreign"
        elif is_broken:
            label = "broken"
        elif is_damaged:
            label = "damaged"
        elif is_chalky:
            label = "chalky"
        results.append({**f,
                        "is_broken": bool(is_broken),
                        "is_chalky": bool(is_chalky),
                        "is_damaged": bool(is_damaged),
                        "is_foreign": bool(is_foreign),
                        "is_normal": bool(is_normal),
                        "label": label})
    meta = {
        "median_len_px": median_len,
        "broken_threshold_px": broken_threshold,
        "chalky_threshold": chalky_threshold,
        "damaged_threshold": damaged_threshold,
        "median_area_px": median_area
    }
    return results, meta

def annotate_polygons(img, results):
    annotated = img.copy()
    colors = {
        "normal": (0, 255, 0),
        "broken": (0, 0, 255),
        "chalky": (0, 165, 255),
        "damaged": (128, 0, 128),
        "foreign": (255, 0, 0)
    }
    for r in results:
        contour = r["contour"]
        label = r.get("label", "normal")
        color = colors.get(label, (255,255,255))
        cv2.drawContours(annotated, [contour], -1, color, 2, cv2.LINE_AA)
        cx, cy = int(r["cx"]), int(r["cy"])
        cv2.circle(annotated, (cx, cy), 2, color, -1)
    return annotated

def pixels_to_mm(avg_pixel_length_px, target_avg_mm=6.0):
    if avg_pixel_length_px <= 0:
        return 0.0
    return float(target_avg_mm / avg_pixel_length_px)

# Streamlit inputs
uploaded = st.file_uploader("Upload a rice image (jpg/png)", type=["jpg","jpeg","png"])
camera_img = st.camera_input("Or capture from your camera")

image_bytes = None
if uploaded is not None:
    image_bytes = uploaded.read()
elif camera_img is not None:
    image_bytes = camera_img.getvalue()

if image_bytes is not None:
    pil = Image.open(BytesIO(image_bytes)).convert("RGB")
    img = np.array(pil)[:, :, ::-1].copy()
    gray, binary = preprocess(img)
    contours = extract_grain_contours(binary, min_area)
    feats = [grain_features(gray, c) for c in contours]
    results, meta = classify_grains(feats, broken_ratio, chalky_percentile, damaged_percentile,
                                    foreign_solidity_thresh, foreign_area_multiplier, foreign_aspect_ratio)

    total = len(results)
    count_broken = sum(1 for r in results if r["label"] == "broken")
    count_chalky = sum(1 for r in results if r["label"] == "chalky")
    count_damaged = sum(1 for r in results if r["label"] == "damaged")
    count_foreign = sum(1 for r in results if r["label"] == "foreign")
    count_normal = sum(1 for r in results if r["label"] == "normal")

    avg_len_px = float(np.mean([r["length_px"] for r in results])) if total else 0.0
    avg_wid_px = float(np.mean([r["width_px"] for r in results])) if total else 0.0
    mm_per_px = pixels_to_mm(avg_len_px, target_avg_mm=6.0) if avg_len_px > 0 else 0.0
    avg_len_mm = avg_len_px * mm_per_px
    avg_wid_mm = avg_wid_px * mm_per_px

    pct_broken = (count_broken / total * 100.0) if total else 0.0
    pct_chalky = (count_chalky / total * 100.0) if total else 0.0
    pct_damaged = (count_damaged / total * 100.0) if total else 0.0
    pct_foreign = (count_foreign / total * 100.0) if total else 0.0
    pct_normal = (count_normal / total * 100.0) if total else 0.0

    col1, col2 = st.columns(2)
    with col1:
        st.subheader("Counts")
        st.metric("Total detected", total)
        st.metric("Normal", f"{count_normal} ({pct_normal:.1f}%)")
        st.metric("Broken", f"{count_broken} ({pct_broken:.1f}%)")
        st.metric("Chalky", f"{count_chalky} ({pct_chalky:.1f}%)")
        st.metric("Damaged/Discoloured", f"{count_damaged} ({pct_damaged:.1f}%)")
        st.metric("Foreign matter", f"{count_foreign} ({pct_foreign:.1f}%)")

        st.subheader("Averages (mm)")
        st.write(f"Average length: **{avg_len_mm:.2f} mm**")
        st.write(f"Average width: **{avg_wid_mm:.2f} mm**")
        st.caption("Calibration: detected avg length mapped to 6.0 mm for conversion. Replace with known mm-per-pixel for accurate absolute sizing.")

    with col2:
        st.subheader("Annotated (polygons)")
        annotated = annotate_polygons(img, results)
        display_img = annotated[:, :, ::-1]
        st.image(display_img, caption="Green=Normal, Red=Broken, Orange=Chalky, Purple=Damaged, Blue=Foreign", use_column_width=True)

        buffer = BytesIO()
        pil_out = Image.fromarray(display_img)
        pil_out.save(buffer, format="PNG")
        buffer.seek(0)
        st.download_button("Download annotated image", data=buffer, file_name="annotated.png", mime="image/png")



else:
    st.info("Upload an image or use the camera to start analysis.")
"""
with open("app.py", "w") as f:
    f.write(app_code)
print("app.py written")


app.py written


In [ ]:

from google.colab import files
# pick a file to upload to /content
uploaded = files.upload()


Saving Gemini_Generated_Image_htq3jxhtq3jxhtq3.png to Gemini_Generated_Image_htq3jxhtq3jxhtq3.png


In [ ]:
import subprocess, time, os, requests
from pyngrok import ngrok

token = os.environ.get("NGROK_AUTHTOKEN", "")
if token:
    ngrok.set_auth_token(token)

# start streamlit
proc = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", "8501", "--server.headless", "true"],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE
)

# wait for local server to respond
start = time.time()

timeout = 40
while True:
    try:
        r = requests.get("http://localhost:8501")
        if r.status_code < 500:
            break
    except:
        pass
    if time.time() - start > timeout:
        print("Warning: Streamlit did not respond within timeout; check logs.")
        break
    time.sleep(0.5)

# create tunnel and print public URL
public_url = ngrok.connect(8501, "http").public_url
print("Public URL:", public_url)
print("If the URL fails, restart this cell. Remove ngrok token cell before sharing notebook.")


Public URL: https://wormlike-magdalen-deductively.ngrok-free.dev
If the URL fails, restart this cell. Remove ngrok token cell before sharing notebook.


In [ ]:
# show logs from the running Streamlit process
import time, sys
for _ in range(200):
    line = proc.stderr.readline()
    if not line:
        time.sleep(0.1)
        continue
    try:
        print(line.decode('utf-8', errors='ignore').rstrip())
    except:
        print(line)
